In [1]:
import numpy as np
import astropy.units as u
import yaml
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord

from gammapy.maps import MapAxis, WcsGeom, Map
from gammapy.data import DataStore
from gammapy.makers import MapDatasetMaker, SafeMaskMaker, FoVBackgroundMaker
from gammapy.modeling.models import (
    FoVBackgroundModel,
    Models,
)
from gammapy.modeling import Fit
from gammapy.datasets import MapDataset
from gammapy.irf import Background3D

import warnings
warnings.filterwarnings('ignore')

In [2]:
muoneff_flag= True
edisp = True

hessera='hess1'
full_runlist = np.loadtxt(f'runlist_{hessera}.txt')

if hessera =='hess1u':
    muoneff_flag=False

In [3]:
# loading general parameters
with open("/home/vault/caph/mppi062h/repositories/HESS_3Dbkg_syserror/general_config.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
conf=cfg['conf']
repo_path=cfg['repo_path']
N_ebins = cfg['N_ebins']
zen_bins = cfg['zen_bins']

muoneff_path = cfg['muoneff_path']
model_str = cfg['model_str']
energy_bins = np.logspace(-1, 2, N_ebins+1)

In [4]:
basedir = f'$FITS_PROD/{hessera}/std_{conf}_fullEnclosure'
ds = DataStore.from_dir(basedir, f'hdu-index-bg-latest-fov-radec.fits.gz', f'obs-index-bg-latest-fov-radec.fits.gz')
obs_table = ds.obs_table

mask = [True if obsid in full_runlist else False for obsid in obs_table['OBS_ID']]
obs_table=obs_table[mask]

In [5]:
axis = MapAxis.from_edges(energy_bins, unit="TeV", name="energy", interp="log")
binsz=cfg['binsz']*u.deg
maker = MapDatasetMaker()
if edisp:
    maker_safe_mask = SafeMaskMaker(methods=['offset-max', 'bkg-peak', 'edisp-bias'], offset_max=cfg['offset_cut']*u.deg) 
else:
    maker_safe_mask = SafeMaskMaker(methods=['offset-max'], offset_max=cfg['offset_cut']*u.deg)

In [6]:
def create_dataset(runlist, axis, muoneff_flag, background_oversampling=1):
    observations = ds.get_observations(runlist)
    maker = MapDatasetMaker()
    geom = WcsGeom.create(skydir=(0,0), binsz=binsz, width=8, frame="galactic", axes=[axis])
    stacked = MapDataset.create(geom=geom)

    for j, obs in enumerate(observations):
        zen_bin = np.sum(obs.pointing_zen.value > zen_bins) - 1
        
        if muoneff_flag:
            muoneff_path=f'/home/saturn/caph/sn0533/shared/hess/fits/bgmodel_3d/prod05/std_zeta_fullEnclosure/{hessera}/hess1_hess2/v01c_kaori_mueff'
            if obs.obs_info['MUONEFF'] > 0.085:
                model_CD = 'B'
            elif obs.obs_info['MUONEFF'] >= 0.075:
                model_CD = 'D'
            else:
                model_CD = 'C'

            if obs.obs_id >= 100000:
                run_number= f'{obs.obs_id}'
            else:
                run_number= f'0{obs.obs_id}'
            filename = f'{muoneff_path}_{model_CD}/hess_bkg_3d_v01c_kaori_mueff_{model_CD}_norebin_fov_radec_{run_number}.fits.gz'
            obs.bkg = Background3D.read(filename, hdu='BACKGROUND')
        
        dataset = stacked.cutout(obs.pointing_radec, width=5)
        dataset = maker.run(dataset, obs)
        
        # this is to set bkg to 0 when it has unreasonable values in the highest energies
        spectrum = np.sum((dataset.background).data, axis=(1,2))
        for i in range(1, 5):
            if spectrum[-1*i] > spectrum[-1*(i+1)]:
                dataset.background.data[-1*i] = 0
        bkg= np.sum((dataset.background).data, axis=(1,2))
        
        dataset = maker_safe_mask.run(dataset, obs)
        
        dataset.mask_fit = Map.from_geom(geom=dataset.counts.geom, data=np.ones_like(dataset.counts.data).astype(bool))   
        dataset.mask_fit &= ~dataset.counts.geom.region_mask(f"galactic;box(0, 0, 4.3, 1.6)")
        dataset.mask_fit &= ~dataset.counts.geom.region_mask(f"galactic;circle(358.71, -0.64, 0.9)")
        
        bkg_model = FoVBackgroundModel(dataset_name=dataset.name)
        dataset.models = Models([bkg_model])
        dataset.background_model.spectral_model.tilt.frozen = False
        Fit().run(datasets=[dataset])
                
        if dataset.background_model.spectral_model.norm.value > 0:
            dataset.background.data[~dataset.mask_safe.data] = 0.0
            stacked.stack(dataset)
        else:
            print(f'run: with problem={obs.obs_id}')

    return stacked

In [7]:
%%time
name = f'20220702_{hessera}_muoneff{muoneff_flag}_edisp{edisp}.fits'
runlist= full_runlist

axis = MapAxis.from_edges(np.logspace(-1,2,25), unit="TeV", name="energy", interp="log")

stacked = create_dataset(runlist, axis, muoneff_flag)
stacked.write(name, overwrite=True)

No HDU found matching: OBS_ID = 20549.0, HDU_TYPE = rad_max, HDU_CLASS = None
No HDU found matching: OBS_ID = 20550.0, HDU_TYPE = rad_max, HDU_CLASS = None
No HDU found matching: OBS_ID = 20551.0, HDU_TYPE = rad_max, HDU_CLASS = None
No HDU found matching: OBS_ID = 20552.0, HDU_TYPE = rad_max, HDU_CLASS = None
No HDU found matching: OBS_ID = 20553.0, HDU_TYPE = rad_max, HDU_CLASS = None
No HDU found matching: OBS_ID = 20554.0, HDU_TYPE = rad_max, HDU_CLASS = None
No HDU found matching: OBS_ID = 20567.0, HDU_TYPE = rad_max, HDU_CLASS = None
No HDU found matching: OBS_ID = 20568.0, HDU_TYPE = rad_max, HDU_CLASS = None
No HDU found matching: OBS_ID = 20572.0, HDU_TYPE = rad_max, HDU_CLASS = None
No HDU found matching: OBS_ID = 20573.0, HDU_TYPE = rad_max, HDU_CLASS = None
No HDU found matching: OBS_ID = 20580.0, HDU_TYPE = rad_max, HDU_CLASS = None
No HDU found matching: OBS_ID = 20581.0, HDU_TYPE = rad_max, HDU_CLASS = None
No HDU found matching: OBS_ID = 20582.0, HDU_TYPE = rad_max, HDU

CPU times: user 28min 29s, sys: 37.2 s, total: 29min 7s
Wall time: 30min 38s
